# 제품 이상여부 판별 프로젝트


In [8]:
import optuna
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)

### 데이터 읽어오기


In [9]:
import pandas as pd

RANDOM_STATE = 110

train_data = pd.read_csv("../../../data/train_data_0816.csv")
test_data = pd.read_csv("../../../data/test_data_0816.csv")

In [10]:
# 공통 변수 리스트
com_variables_train = [
    'target', 'Model.Suffix', 'Workorder', 'WorkMode Collect Result'
    , 'Dispenser_1', 'Dispenser_2', 'Receip_No_Collect_Result'
    , 'Production_Qty_Collect_Result', 'Judge_Value_OK'
    , 'Workorder_0.9', 'Workorder_0.6'
]

com_variables_test = [
    'target', 'Set ID', 'Model.Suffix', 'Workorder'
    , 'WorkMode Collect Result', 'Dispenser_1'
    , 'Dispenser_2', 'Receip_No_Collect_Result'
    , 'Production_Qty_Collect_Result', 'Judge_Value_OK'
    , 'Workorder_0.9', 'Workorder_0.6'
]

In [11]:
# 공정 이름 필터링 후 공통 변수와 결합
def create_dataset(train_data, test_data, process_name, com_variables_train, com_variables_test):
    # 열 이름 필터링
    Process_Desc_col = train_data.filter(like=process_name).columns
    
    # train 데이터셋 생성
    final_columns_train = list(Process_Desc_col) + com_variables_train
    train_dataset = train_data[final_columns_train]
    
    # test 데이터셋 생성
    final_columns_test = list(Process_Desc_col) + com_variables_test
    test_dataset = test_data[final_columns_test]
    
    return train_dataset, test_dataset

# 공통 변수 정의
## com_variables_train = [...]  -> 이전 코드에서 정의한 변수 사용
## com_variables_test = [...]   -> 이전 코드에서 정의한 변수 사용

# 데이터셋 생성
train_data_dam, test_data_dam = create_dataset(train_data, test_data, '_Dam', com_variables_train, com_variables_test)
train_data_fill1, test_data_fill1 = create_dataset(train_data, test_data, '_Fill1', com_variables_train, com_variables_test)
train_data_fill2, test_data_fill2 = create_dataset(train_data, test_data, '_Fill2', com_variables_train, com_variables_test)
train_data_autoclave, test_data_autoclave = create_dataset(train_data, test_data, '_AutoClave', com_variables_train, com_variables_test)

---

## Optuna

스레스홀드 0.3으로 맞춘상태에서 튜닝 진행한 것

In [12]:
# import optuna
# from lightgbm import LGBMClassifier
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import f1_score

# # 'Normal'과 'AbNormal'을 숫자로 변환
# train_data_fill1['target'] = train_data_fill1['target'].map({'Normal': 0, 'AbNormal': 1})

# # 스레드홀드 설정
# THRESHOLD = 0.3

# def objectiveLGBM_dart(trial, x_tr, y_tr, x_val, y_val):
#     param = {
#         'n_estimators': trial.suggest_int('n_estimators', 500, 3000),
#         'num_leaves': trial.suggest_int('num_leaves', 500, 3000),
#         'max_depth': trial.suggest_int('max_depth', 10, 300),
#         'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
#         'min_child_samples': trial.suggest_int('min_child_samples', 3, 300),
        
#         'boosting_type': 'dart',  # 'boosting'를 'boosting_type'으로 수정
#         'random_state': RANDOM_STATE,
#         'verbose': -1
#     }
       
#     model = LGBMClassifier(**param)
#     model.fit(x_tr, y_tr)
#     pred_proba = model.predict_proba(x_val)[:, 1]  # 양성 클래스 확률
#     pred = (pred_proba >= THRESHOLD).astype(int)  # 스레드홀드에 따른 예측
    
#     score = f1_score(y_val, pred, average="binary")
    
#     return score

# # 데이터셋 분할
# x_train, x_val, y_train, y_val = train_test_split(
#     train_data_fill1.drop("target", axis=1),
#     train_data_fill1["target"],
#     test_size=0.2,
#     shuffle=True,
#     random_state=RANDOM_STATE,
# )

# # 하이퍼 파라미터 튜닝
# study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=RANDOM_STATE))
# study.optimize(lambda trial: objectiveLGBM_dart(trial, x_train, y_train, x_val, y_val), n_trials=100)

# print('Best trial: score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))


Best trial: score 0.24512534818941506,   
params {'n_estimators': 691, 'num_leaves': 555, 'max_depth': 74,  
'learning_rate': 0.051907937588647424, 'min_child_samples': 59}  

In [19]:
# from lightgbm import LGBMClassifier
# from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, confusion_matrix
# from sklearn.model_selection import train_test_split

# THRESHOLD = 0.3

# # 최적의 하이퍼파라미터
# best_params = {
#     'n_estimators': 691,
#     'num_leaves': 555,
#     'max_depth': 74,
#     'learning_rate': 0.051907937588647424,
#     'min_child_samples': 59,
#     'boosting_type': 'dart',
#     'random_state': RANDOM_STATE,
#     'verbose': -1
# }

# # 모델 생성
# model = LGBMClassifier(**best_params)

# # 데이터셋 분할 (재사용)
# x_train, x_val, y_train, y_val = train_test_split(
#     train_data_fill1.drop("target", axis=1),
#     train_data_fill1["target"],
#     test_size=0.2,
#     shuffle=True,
#     random_state=RANDOM_STATE,
# )

# # y_train과 y_val을 이진 레이블로 변환
# y_train = y_train.apply(lambda x: 1 if x == 'AbNormal' else 0)
# y_val = y_val.apply(lambda x: 1 if x == 'AbNormal' else 0)

# # 모델 학습
# model.fit(x_train, y_train)

# # 검증 데이터에 대한 예측
# y_val_pred_proba = model.predict_proba(x_val)[:, 1]  # 양성 클래스 확률
# y_val_pred = (y_val_pred_proba >= THRESHOLD).astype(int)  # 스레드홀드에 따른 예측

# # 평가지표 계산
# f1 = f1_score(y_val, y_val_pred, average="binary")
# accuracy = accuracy_score(y_val, y_val_pred)
# precision = precision_score(y_val, y_val_pred)
# recall = recall_score(y_val, y_val_pred)

# # 혼동 행렬 계산
# conf_matrix = confusion_matrix(y_val, y_val_pred)

# # 결과 출력
# print(f'F1 Score: {f1}')
# print('---')
# print('Confusion Matrix:')
# print(conf_matrix)  # 혼동 행렬 출력
# print('---')
# print(f'Accuracy: {accuracy}')
# print(f'Precision: {precision}')
# print(f'Recall: {recall}')

F1 Score: 0.24512534818941506
---
Confusion Matrix:
[[7472  190]
 [ 352   88]]
---
Accuracy: 0.9331029375462848
Precision: 0.31654676258992803
Recall: 0.2


In [14]:
# from xgboost import XGBClassifier
# from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, confusion_matrix
# from sklearn.model_selection import train_test_split

# THRESHOLD = 0.3

# # 최적의 하이퍼파라미터
# best_params = {
#     'n_estimators': 910, 
#     'learning_rate': 0.1516322549932977, 
#     'max_depth': 20, 
#     'alpha': 0.003953220971450215, 
#     'gamma': 0.0013290323576843753, 
#     'reg_alpha': 0.35046775966926846, 
#     'reg_lambda': 0.020721087884783395, 
#     'colsample_bytree': 0.9070762583460225, 
#     'subsample': 0.8592284053859449
# }

# # 모델 생성
# model = XGBClassifier(**best_params)

# # 데이터셋 분할 (재사용)
# x_train, x_val, y_train, y_val = train_test_split(
#     train_data_dam.drop("target", axis=1),
#     train_data_dam["target"],
#     test_size=0.2,
#     shuffle=True,
#     random_state=RANDOM_STATE,
# )

# # y_train과 y_val을 이진 레이블로 변환
# y_train = y_train.apply(lambda x: 1 if x == 'AbNormal' else 0)
# y_val = y_val.apply(lambda x: 1 if x == 'AbNormal' else 0)

# # 모델 학습
# model.fit(x_train, y_train)

# # 검증 데이터에 대한 예측
# y_val_pred_proba = model.predict_proba(x_val)[:, 1]  # 양성 클래스 확률
# y_val_pred = (y_val_pred_proba >= THRESHOLD).astype(int)  # 스레드홀드에 따른 예측

# # 평가지표 계산
# f1 = f1_score(y_val, y_val_pred, average="binary")
# accuracy = accuracy_score(y_val, y_val_pred)
# precision = precision_score(y_val, y_val_pred)
# recall = recall_score(y_val, y_val_pred)

# # 혼동 행렬 계산
# conf_matrix = confusion_matrix(y_val, y_val_pred)

# # 결과 출력
# print(f'F1 Score: {f1}')
# print('---')
# print('Confusion Matrix:')
# print(conf_matrix)  # 혼동 행렬 출력
# print('---')
# print(f'Accuracy: {accuracy}')
# print(f'Precision: {precision}')
# print(f'Recall: {recall}')

F1 Score: 0.19084967320261442
---
Confusion Matrix:
[[7410  234]
 [ 385   73]]
---
Accuracy: 0.9235991113305356
Precision: 0.23778501628664495
Recall: 0.15938864628820962


.